<a href="https://colab.research.google.com/github/shris-ai/genai-implementations/blob/main/Sarvam_Parse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation & Setup

## Install required libraries

In [ ]:
!pip install requests
!pip install requests-toolbelt

## Download data

In [ ]:
!gdown https://drive.google.com/uc?id=1bCpso1rDbRY3srTDjfvYhLrq8hQkFAqP

Downloading...
From: https://drive.google.com/uc?id=1bCpso1rDbRY3srTDjfvYhLrq8hQkFAqP
To: /content/Prompt Engineering by Lilian Weng.pdf
100% 4.11M/4.11M [00:00<00:00, 64.9MB/s]


## Get secrets from Colab

In [ ]:
from google.colab import userdata
api_key = userdata.get('sarvam_api_key')

# Sarvam Parse

As Sarvam Parse accepts document data in the form of multipart form data, we will use Multipart Encoder to convert required field into acceptable format.

In [ ]:
from requests_toolbelt.multipart.encoder import MultipartEncoder


path = r"/content/Prompt Engineering by Lilian Weng.pdf"

payload = MultipartEncoder(
    fields={
        "pdf": ('Prompt Engineering by Lilian Weng.pdf', open(path, 'rb'),  'application/pdf'),
        "page_number": "1",
        "sarvam_mode": "small",
        "prompt_caching": "true"
        }
    )


Our payload is ready. Let's send a POST request to Sarvam's endpoint for Sarvam Parse.

In [ ]:
import requests

url = "https://api.sarvam.ai/parse/parsepdf"

headers = {
    "api-subscription-key": api_key,
    "Content-Type": payload.content_type
    }

response = requests.request('POST', url, data=payload, headers=headers)

print(response.status_code)
print(response.json())


200
{'output': 'PCFET0NUWVBFIGh0bWw+CjxodG1sIGxhbmc9ImVuIj4KICA8aGVhZD4KICAgIDxtZXRhIGNoYXJzZXQ9IlVURi04IiAvPgogICAgPG1ldGEgbmFtZT0idmlld3BvcnQiIGNvbnRlbnQ9IndpZHRoPWRldmljZS13aWR0aCwgaW5pdGlhbC1zY2FsZT0xLjAiIC8+CiAgPC9oZWFkPgogIDxib2R5PgogICAgPHBhZ2UtaGVhZGVyIGJib3g9IlsyMy4wLCAxNC4wLCAzOTIuMCwgMjYuMF0iPgozLzgvMjUsIDc6MjcgQU0gUHJvbXB0IEVuZ2luZWVyaW5nIHwgTGlsJ0xvZwo8L3BhZ2UtaGVhZGVyPgo8cGFnZS1oZWFkZXIgYmJveD0iWzIyNy4wLCA0NC4wLCA1NTIuMCwgNjAuMF0iPgpQb3N0cyBBcmNoaXZlIFNlYXJjaCBUYWdzIEZBUSBlbW9qaXNlYXJjaC5hcHAKPC9wYWdlLWhlYWRlcj4KPGNoYXB0ZXItdGl0bGU+ClByb21wdCBFbmdpbmVlcmluZwo8L2NoYXB0ZXItdGl0bGU+CjxwIGJib3g9Ils0Ni4wLCAxNDUuMCwgNDAyLjAsIDE1OS4wXSI+CkRhdGU6IE1hcmNoIDE1LCAyMDIzIHwgRXN0aW1hdGVkIFJlYWRpbmcgVGltZTogMjEgbWluIHwgQXV0aG9yOiBMaWxpYW4gV2VuZwo8L3A+Cjx0b2MgYmJveD0iWzY4LjAsIDE4NC4wLCAxNzAuMCwgMTk2LjBdIj4KVGFibGUgb2YgQ29udGVudHMKPC90b2M+CjxwIGJib3g9Ils0Ni4wLCAyMzguMCwgNTQ2LjAsIDMxMC4wXSI+CjxiPlByb21wdCBFbmdpbmVlcmluZzwvYj4sIGFsc28ga25vd24gYXMgPGI+SW4tQ29udGV4dCBQcm9tcHRpbmc8L2I+LCByZWZl

Since, the API's status code is 200, we'll proceed further.


The output of the API is a base64 encoded XML string containing the extracted data.

The final step here is to decode the XML string.

In [ ]:
import base64
import xml.etree.ElementTree as ET

base64_encoded_xml = (response.json())['output']

decoded_xml = base64.b64decode(base64_encoded_xml).decode('utf-8')

print(decoded_xml)

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
  </head>
  <body>
    <page-header bbox="[23.0, 14.0, 392.0, 26.0]">
3/8/25, 7:27 AM Prompt Engineering | Lil'Log
</page-header>
<page-header bbox="[227.0, 44.0, 552.0, 60.0]">
Posts Archive Search Tags FAQ emojisearch.app
</page-header>
<chapter-title>
Prompt Engineering
</chapter-title>
<p bbox="[46.0, 145.0, 402.0, 159.0]">
Date: March 15, 2023 | Estimated Reading Time: 21 min | Author: Lilian Weng
</p>
<toc bbox="[68.0, 184.0, 170.0, 196.0]">
Table of Contents
</toc>
<p bbox="[46.0, 238.0, 546.0, 310.0]">
<b>Prompt Engineering</b>, also known as <b>In-Context Prompting</b>, refers to methods for how to communicate with LLM to steer its behavior for desired outcomes <i>without</i> updating the model weights. It is an <i>empirical</i> science and the effect of prompt engineering methods can vary a lot among models, thus requiring heavy e

If we want to extract all the paragraphs...

In [ ]:
# Example: Extracting all paragraphs
root = ET.fromstring(decoded_xml)

for paragraph in root.iter('p'):
  print(paragraph.text)


Date: March 15, 2023 | Estimated Reading Time: 21 min | Author: Lilian Weng




This post 

[

Zero-shot and few-shot learning are two most basic approaches for prompting the model, pioneered by many LLM papers and commonly used for benchmarking LLM performance.




(All the sentiment analysis examples are from SST-2)


Text: i'll bet the video game is a lot more fun than the film.
Sentiment:



That's all folks! 😼